# Should I call 911?!

In [1]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series
import psycopg2
from sqlalchemy import create_engine
import pickle

# Dataset:  Safety-Conditions

Source: Dataset defined by Safety Expert based on literature and experience

Description: Safety risk definition based on situation characteristics.

Variables/Columns
PERCEPTION: Own Safety level perception from 1 (very safe)to 5 (very unsafe)
ARMMED: Binary, is there an armmed person near to you that could harm you?
COMMUNICATION: Binary, are you able to call someone or ask for help in a safe manner?
FOLLOWED: Binary, are you being followed?
THREAT: Binary, do you feel your life is in danger?
AREA: Binary, are you in a familar area?
ILLUMMINATION: Perceived illummination level in area from 1 (poorly illuminated) to 3 (totally illuminated)
ACCOMPANNIED: Binary, are you accompanied?

In [2]:
# Connection parameters
param_dic = {
    "host"      : "ec2-54-237-135-248.compute-1.amazonaws.com",
    "database"  : "d5acm9llef4cc3",
    "user"      : "ixthtifqgyfsbs",
    "password"  : "043f429de1d27aff7dc4f86cadc51d8d08e8defef0b0727db44f02f61f5bc1ff"
}

In [3]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    print("Connection successful")
    return conn

In [4]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [5]:
def single_insert(conn, insert_req):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [6]:
# Connect to the database
conn = connect(param_dic)
column_names = ["perception", "armmed", "communication","followed","threat","area","ilummination","accompannied","danger"]
# Execute the "SELECT *" query
conditions = postgresql_to_dataframe(conn, "select * from danger", column_names)
conditions.head()

Connecting to the PostgreSQL database...
Connection successful


,perception,armmed,communication,followed,threat,area,ilummination,accompannied,danger
0,3,Si,Si,No,Si,Si,3,No,TRUE
1,2,No,No,Si,Si,Si,3,Si,FALSE
2,5,No,No,No,Si,No,1,No,TRUE
3,1,No,Si,No,No,Si,3,Si,FALSE
4,4,No,No,No,Si,No,1,No,TRUE


In [7]:
conditions.rename(columns={"perception":"PERCEPTION","armmed":"ARMMED","communication":"COMMUNICATION","followed":"FOLLOWED","threat":"THREAT","area":"AREA","ilummination":"ILLUMMINATION","accompannied":"ACCOMPANNIED","danger":"DANGER"},inplace=True)
X = conditions[["PERCEPTION", "ARMMED", "COMMUNICATION","FOLLOWED","THREAT","AREA","ILLUMMINATION","ACCOMPANNIED"]]
y = conditions["DANGER"].values.reshape(-1, 1)
print(X.shape, y.shape)

(151, 8) (151, 1)


In [8]:
data = X.copy()

data_binary_encoded = pd.get_dummies(data, columns=["ARMMED", "COMMUNICATION","FOLLOWED","THREAT","AREA","ACCOMPANNIED"])
data_binary_encoded.drop(['ARMMED_No','ARMMED_No se','COMMUNICATION_No','FOLLOWED_No','THREAT_No','AREA_No','ACCOMPANNIED_No'], axis=1, inplace=True)
data_binary_encoded.head()

,PERCEPTION,ILLUMMINATION,ARMMED_Si,COMMUNICATION_Si,FOLLOWED_Si,THREAT_Si,AREA_Si,ACCOMPANNIED_Si
0,3,3,1,1,0,1,1,0
1,2,3,0,0,1,1,1,1
2,5,1,0,0,0,1,0,0
3,1,3,0,1,0,0,1,1
4,4,1,0,0,0,1,0,0


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_binary_encoded, y, random_state=42)

X_train.head()

,PERCEPTION,ILLUMMINATION,ARMMED_Si,COMMUNICATION_Si,FOLLOWED_Si,THREAT_Si,AREA_Si,ACCOMPANNIED_Si
105,2,3,0,1,1,1,0,0
4,4,1,0,0,0,1,0,0
32,2,3,0,1,1,1,0,0
42,3,2,0,0,0,1,0,0
144,2,3,0,0,0,1,1,1


In [10]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [11]:
classifier.fit(X_train, y_train)

LogisticRegression()

In [12]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9292035398230089
Testing Data Score: 0.9210526315789473


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracyScore = accuracy_score(y_test, predictions)
precisionScoreTRUE = precision_score(y_test, predictions, pos_label="TRUE")
precisionScoreFALSE = precision_score(y_test, predictions, pos_label="FALSE")
recallScoreTRUE = recall_score(y_test, predictions, pos_label="TRUE")
recallScoreFALSE = recall_score(y_test, predictions, pos_label="FALSE")
f1ScoreTRUE = f1_score(y_test, predictions, pos_label="TRUE")
f1ScoreFALSE = f1_score(y_test, predictions, pos_label="FALSE")

metrics = {'Metrics': ["Accuracy", "In Danger Precision", "Not in Danger Precision","In Danger Recall", "Not in Danger Recall", "In Danger F1-Score","Not in Danger F1-Score"], 'Score': [accuracyScore, precisionScoreTRUE, precisionScoreFALSE, recallScoreTRUE, recallScoreFALSE, f1ScoreTRUE, f1ScoreFALSE]}
metricsDF = pd.DataFrame(data=metrics)
metricsDF.to_csv(r'\Resources\metrics.csv', index = False)
metricsDF

,Metrics,Score
0,Accuracy,0.921053
1,In Danger Precision,0.863636
2,Not in Danger Precision,1.000000
3,In Danger Recall,1.000000
4,Not in Danger Recall,0.842105
5,In Danger F1-Score,0.926829
6,Not in Danger F1-Score,0.914286


In [28]:
filename = 'finalized_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [23]:
import numpy as np
new_data = np.array([[1,2,0,1,1,1,0,1]])
predictions = classifier.predict(new_data)
print("Classes are either False (Not in Critical Danger) or True (In Danger)")
print(f"The new point was classified as: {predictions}")

Classes are either False (Not in Critical Danger) or True (In Danger)
The new point was classified as: ['FALSE']


In [13]:
predictions = classifier.predict(X_test)
sample = pd.DataFrame(y_test)
sample.rename(columns={0: "Actual"}, inplace=True)
sample['Predictions'] = predictions
sample

,Actual,Predictions
0,FALSE,FALSE
1,TRUE,TRUE
2,FALSE,FALSE
3,TRUE,TRUE
4,TRUE,TRUE
5,FALSE,FALSE
6,TRUE,TRUE
7,TRUE,TRUE
8,TRUE,TRUE
9,FALSE,FALSE


In [35]:
print("Hi! This is your 911 Chatbot. I'll help you decide if according to the situation you are in, you should call 911.")
print("REMEMBER! If you're already in a critical situation, CALL 911!")

SAFE = int(input("How safe do you feel? Write a number from 1 (very safe) to 5 (in danger) "))
LIGHT = int(input("Is the are where you are well illuminated? Write 1 if Poorly, 2 if Partially, 3 if Well Illuminated "))
ARM = int(input("Do you see someone with an arm that could harm you? Write 1 if Yes, 0 if No "))
COMM = int(input("Are you able to call for help? Write 1 if Yes, 0 if No "))
FOLLOW = int(input("Are you being followed? Write 1 if Yes, 0 if No "))
THREAT = int(input("Do you feel your life is in danger? Write 1 if Yes, 0 if No "))
AREA = int(input("Are you familiar with the area you are in currently? Write 1 if Yes, 0 if No "))
COMPANY = int(input("Are you accompanied? Write 1 if Yes, 0 if No "))
answers = np.array([[SAFE,LIGHT,ARM,COMM,FOLLOW,THREAT,AREA,COMPANY]])
yourPrediction = classifier.predict(answers)

if yourPrediction == "FALSE":
    print("Please don't call 911! You should be able to handle the situation, ask for help and take care of yourself")
else:
    print("CALL 911 INMMEDIATELY! An officer should be able to help you")
    DNGRresp = int(input("FOLLOW UP QUESTION!! Ask your officer, were you assisted in a dangerous situation? Write 1 if Yes, 0 if No "))

if DNGRresp == 1:
    DNGR = "TRUE"
else:
    DNGR = "FALSE"

Hi! This is your 911 Chatbot. I'll help you decide if according to the situation you are in, you should call 911.
REMEMBER! If you're already in a critical situation, CALL 911!
How safe do you feel? Write a number from 1 (very safe) to 5 (in danger) 5
Is the are where you are well illuminated? Write 1 if Poorly, 2 if Partially, 3 if Well Illuminated 1
Do you see someone with an arm that could harm you? Write 1 if Yes, 0 if No 1
Are you able to call for help? Write 1 if Yes, 0 if No 0
Are you being followed? Write 1 if Yes, 0 if No 1
Do you feel your life is in danger? Write 1 if Yes, 0 if No 1
Are you familiar with the area you are in currently? Write 1 if Yes, 0 if No 0
Are you accompanied? Write 1 if Yes, 0 if No 0
CALL 911 INMMEDIATELY! An officer should be able to help you
FOLLOW UP QUESTION!! Ask your officer, were you assisted in a dangerous situation? Write 1 if Yes, 0 if No 1


In [38]:
if ARM == 1:
    ARMdb = "Si"
else:
    ARMdb = "No"

if COMM == 1:
    COMMdb = "Si"
else:
    COMMdb = "No"
    
if FOLLOW == 1:
    FOLLOWdb = "Si"
else:
    FOLLOWdb = "No"
    
if THREAT == 1:
    THREATdb = "Si"
else:
    THREATdb = "No"

if AREA == 1:
    AREAdb = "Si"
else:
    AREAdb = "No"
    
if COMPANY == 1:
    COMPANYdb = "Si"
else:
    COMPANYdb = "No"

In [41]:
conn = connect(param_dic)
# Build the insert query
query = """INSERT into danger (perception,armmed,communication,followed,threat,area,ilummination,accompannied,danger) values ('%s','%s','%s','%s','%s','%s','%s','%s','%s');""" % (SAFE,ARMdb,COMMdb,FOLLOWdb,THREATdb,AREAdb,LIGHT,COMPANYdb,DNGR)
# Insert into the database
single_insert(conn, query)

Connecting to the PostgreSQL database...
Connection successful
